<a href="https://colab.research.google.com/github/Ranjan4Kumar/Adaptive_Online_Education/blob/main/Student_Adaptivity_online_Edu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! pip install -q kaggle

In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/

# ML Prediction on Student's Adaptability Level in Online Education.
Data is collected according here are student's information on three levels school, colleges, Universities.

By conducting bith online and physical survey.


#### 13 features and 1 target columns in the dataset, 
1 -> Gender : Gender type of student <br>
2 -> Age : Age range of the student <br>
3 -> Educational Level : Educaton intitution level<br>
4 -> Instituion Type : Education institution type <br>
5 -> IT Student : Studying as IT student or not <br>
6 -> Location : Weather student is located in town or not <br>
7 -> Load-Shedding : Level of Load Shedding <br>
8 -> Financial condition of family <br>
9 -> Internet Type : Internet type used mostly in device <br>
10 -> Network type : Network connectivitiy type <br>
11 -> Class Duration : Daily Class duration <br>
12 -> Self LMS : Instituion's own LMS availability <br>
13 -> Device : Device used mostly in class <br>

And taget column is <br>
14 -> Adaptability Level : Adaptability level of student

### Notebook Objectives
(1) Univariate & Multivariate Analysis on features
(2) Data Preprocessing for ML Prediction
(3) ML Model Comparison [part 1]
(4) Model Selection & Hyper parameter Tuning
(5) ML Model Comparison [part2]


In [ ]:
# !kaggle kernels output wonduk/predict-eda-on-adaptivity-in-online-education -p /path/to/dest

In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 8.4 MB/s eta 0:00:00


In [2]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.5 MB/s eta 0:00:00


# 1.Import Necessary Libraries

In [3]:
import numpy as np
import pandas as pd

# For visulization
import seaborn as sns
import matplotlib as mpl
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Modeling Libraries

from xgboost import XGBClassifier
from catboost import CatBoostClassifier
'''
 => CatBoost is an open-source gradient boosting library for decision trees developed by Yandex. 
It is designed to handle categorical data efficiently and is particularly useful for working with datasets that contain a large number of categorical features. 
Some of the key features of CatBoost include:

1 -> Handling of categorical features: CatBoost can handle categorical features without the need to one-hot encode them, which can save memory and improve performance.

2 -> Handling of missing values: CatBoost can handle missing values in the input data without the need for imputation.

3 -> Handling of skewed data: CatBoost can handle data with skewed class distributions, which is common in many real-world datasets.

4 -> Handling of overfitting: CatBoost includes built-in overfitting detection and can automatically stop training when overfitting occurs.

5 -> Handling of different types of data: CatBoost supports a variety of data types, including numerical, categorical, and ordinal.

6 -> Multi-language support: CatBoost has interfaces for Python, R, and command-line interfaces.

7 -> CatBoost is designed to work well with large datasets and can be used for both regression and classification tasks. 
It also provides an easy-to-use API and can be integrated with other libraries such as pandas, scikit-learn, and numpy.
'''
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Cluster Libraries
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import KElbowVisualizer

# Tuning Libraries
import optuna
'''
Optuna is an open-source hyperparameter optimization framework for machine learning. It provides an easy-to-use interface to perform optimization tasks such as tuning the 
hyperparameters of machine learning models. Optuna uses various optimization algorithms, such as Tree-structured Parzen Estimator (TPE) and Random Search, to find the best set
of hyperparameters for a given model. Additionally, Optuna can integrate with other machine learning libraries, such as TensorFlow, PyTorch, and scikit-learn, 
to make it easy to use in a variety of projects.
'''

# Metrics & Scoring Libraries
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Options

%matplotlib inline
from termcolor import colored
plt.rcParams['axes.unicode_minus'] = False
pd.reset_option('display.float_format')
pd.set_option('display.max_columns', None)

color_scheme = px.colors.qualitative.Pastel

In [4]:
color_scheme

['rgb(102, 197, 204)',
 'rgb(246, 207, 113)',
 'rgb(248, 156, 116)',
 'rgb(220, 176, 242)',
 'rgb(135, 197, 95)',
 'rgb(158, 185, 243)',
 'rgb(254, 136, 177)',
 'rgb(201, 219, 116)',
 'rgb(139, 224, 164)',
 'rgb(180, 151, 231)',
 'rgb(179, 179, 179)']

In [5]:
'''
Define bar plot using Plotly

'''
# def p_bar_plot(x,y,x_axis_title, y_axis_title, title):
#   fig = px.bar(x = x, y = y, color = x, text = y, color_discrete_sequence = color_scheme, template = 'simple_white')
#   fig_update_layout(xaxis_title = x_axis_title, yaxis = y_axis_title, title = title, font = dict(size = 17, family = "Franklin Gothic"))
#   fig.update_traces(width = 0.5)
#   fig.show()

def p_bar_plot(x, y, x_axis_title, y_axis_title, title):
    fig = px.bar(x = x, y = y, color = x, text = y, color_discrete_sequence = color_scheme, template = 'simple_white')
    fig.update_layout(xaxis_title = x_axis_title, yaxis_title= y_axis_title, title = title, font = dict(size=17,family="Franklin Gothic"))
    fig.update_traces(width=0.5)
    fig.show()


In [6]:
'''
Define pie plot using Plotly
'''

# def p_pie_plot(series,title):
#   fig = px.pie(values = series.values, names = series.index, color_discrete_sequence=color_scheme, title = title, template = 'simple_white')
#   fig_update_traces(textinfo = 'label+percent+value', textfont_size = 20,
#                      marker = dict(line = dict(color = '#100000', width = 0.2)))
#   fig.data[0].marker.line.width = 1
#   fig.update_layout(font = dict(size=20, family = 'Franklin Gothic'))
#   fig.show()

def p_pie_plot(series, title):
    fig = px.pie(values=series.values, names=series.index, color_discrete_sequence=color_scheme,title= title, template='simple_white')
    fig.update_traces(textinfo='label+percent+value', textfont_size=20,
                  marker=dict(line=dict(color='#100000', width=0.2)))
    fig.data[0].marker.line.width = 1
    fig.update_layout(font=dict(size=20,family="Franklin Gothic"))
    fig.show()

We will laod data 

In [7]:
df = pd.read_csv('/content/drive/MyDrive/DS/Predict & EDA on Adaptivity in Online Education/students_adaptability_level_online_education.csv')

print(colored('Shape of DataFrame:', 'blue'), df.shape, '\n\n')
df.head()

Shape of DataFrame: (1205, 14) 




,Gender,Age,Education Level,Institution Type,IT Student,Location,Load-shedding,Financial Condition,Internet Type,Network Type,Class Duration,Self Lms,Device,Adaptivity Level
0,Boy,21-25,University,Non Government,No,Yes,Low,Mid,Wifi,4G,3-6,No,Tab,Moderate
1,Girl,21-25,University,Non Government,No,Yes,High,Mid,Mobile Data,4G,1-3,Yes,Mobile,Moderate
2,Girl,16-20,College,Government,No,Yes,Low,Mid,Wifi,4G,1-3,No,Mobile,Moderate
3,Girl,11-15,School,Non Government,No,Yes,Low,Mid,Mobile Data,4G,1-3,No,Mobile,Moderate
4,Girl,16-20,School,Non Government,No,Yes,Low,Poor,Mobile Data,3G,0,No,Mobile,Low


# 2. Explore Data Analysis
Mainly 2 parts in 'Explorer Data Analysis' section, each are: * Univariate Analysis * Multivariate Analysis

# 2.1 Univariate Analysis

#### 1 ) Gender : Gender type of student <br>

#### 2 ) Institution Type : Education institution type

In [8]:
list_1 = []
list_1_title = ['Gender' , 'Institution Type']
list_1.append(df['Gender'].value_counts())
list_1.append(df['Institution Type'].value_counts())

fig = make_subplots(rows = 1, cols = 2, specs = [[{"type":"pie"},{"type":"pie"}]])
'''
The enumerate() function in Python is a built-in function that allows you to iterate over a sequence (such as a list, tuple, or string)
and keep track of the index of the current item

It takes an iterable object as an input and returns an iterator that produces tuples containing the index and the corresponding item from the input.

Ex-> 
fruits = ['apple', 'banana', 'orange']
for index, fruit in enumerate(fruits, start=1):
    print(index, fruit)

output
1 apple
2 banana
3 orange

'''
for idx, series in enumerate(list_1):
  fig.add_trace(go.Pie(values = series.values,
                       labels = series.index,
                       marker = dict(colors = color_scheme),
                       title = str(idx+1)+ '.' + list_1_title[idx]),
                       row = 1,
                       col = idx+1 )
  fig.update_traces(textinfo = 'label+percent+value', textfont_size = 13,
  marker = dict(line=dict(color = '#100000', width = 0.2)))

fig.show()


#### 3 ) Age and 4 ) Class Dusration

3 ) Age : Age range of the student <br>
4 ) class duration : Daily class duration

In [9]:
age = df['Age'].value_counts().sort_index()
class_duration = df['Class Duration'].value_counts().sort_index()
p_bar_plot(age.index , age.values , 'Age' , 'Count' , '3. Age Distribution')
p_bar_plot(class_duration.index , class_duration.values, 'Class Duration', 'Count', '4. Class Duration')

Age Of the respondents is mainly distributed between 11 and 25 <br>
Class Duration mainly distributed between 1-3 hrs



#### 5. Education Level & 6. Financial Condition & 7.Network Type

> 5) Education Level: Education institution level <br>
6) Financial Condition : Family financail Condition<br>
7) Netwrok Type : Network connectivity type

In [10]:
list_2 = []
list_2_title = ['Education Level' , ' Financial Condition' , 'Netwrok Type']
list_2.append(df['Education Level'].value_counts())
list_2.append(df['Financial Condition'].value_counts())
list_2.append(df['Network Type'].value_counts())

fig = make_subplots(rows = 1 , cols = 3, specs = [[{"type":"pie"},{"type":"pie"},{"type":"pie"}]])

for idx, series in enumerate(list_2):
  fig.add_trace(go.Pie(values = series.values,
                       labels = series.index,
                       marker = dict(colors = color_scheme),
                       title = str(idx+1)+ '.' + list_2_title[idx]),
                       row = 1,
                       col = idx+1 )
  fig.update_traces(textinfo = 'label+percent+value', textfont_size = 13,
  marker = dict(line=dict(color = '#100000', width = 0.2)))

fig.show()

Most of the repondents attend School and University <br>
And most of them are situated in Mid financial condition <br>
There are just 1.58% of students using 2G network

#### 8. IT Students & 9. Location & 10.Self LMS
> 8. It student: Studying as IT student or not
9. Location: Is student located in town
10. Self Lms: Institution's own LMS availability(Learning Managment System)

In [11]:
list_3 = []
list_3_title = ['IT Student' , 'Location', 'Self Lms']
list_3.append(df['IT Student'].value_counts())
list_3.append(df['Location'].value_counts())
list_3.append(df['Self Lms'].value_counts())
fig = make_subplots(rows = 1 , cols = 3 , specs = [[{"type":"pie"},{"type":"pie"},{"type":"pie"}]])

for idx, series in enumerate(list_3):
  fig.add_trace(go.Pie(values = series.values,
                       labels = series.index,
                       marker = dict(colors = color_scheme),
                       title = str(idx+1)+'.'+ list_3_title[idx]),
                       row = 1,
                       col = idx+1)
  fig.update_traces(textinfo = 'label+percent+value', textfont_size = 13,
                    marker = dict(line = dict(color = '#100000', width = 0.2)))
fig.show()

About 25.2% of respondents ares tudying as IT student <br>
And 77.6% of them are located in town <br>
Furthermore, 17.4 of their instructions own LMS availability

#### 11. Load-shadding 12. Internet Type 13. Device
> 11. Load-Shadding : Level of load shedding
12. Internet Type : Internet type used mostly in device
13. Device : Device used mostly in class



In [12]:
list_4 = []
list_4_title = ['Load-shedding','Internet Type','Device']
list_4.append(df['Load-shedding'].value_counts())
list_4.append(df['Internet Type'].value_counts())
list_4.append(df['Device'].value_counts())

fig = make_subplots(rows = 1, cols = 3, specs = [[{'type':'pie'},{'type':'pie'},{'type':'pie'}]])

for idx, series in enumerate(list_4):
  fig.add_trace(go.Pie(values = series.values,
                       labels = series.index,
                       marker = dict(colors = color_scheme),
                       title = str(idx+1)+'.'+ list_4_title[idx]),
                row = 1,
                col = idx+1)
  fig.update_traces(textinfo = 'label+percent+value', textfont_size = 13,
                    marker = dict(line = dict(color='#100000', width = 0.2)))
fig.show()

Load-shedding refers to level of load shedding, and type 'low' account for 83.3% <br>

57.7% of respondents use mobile data to take online classes <br>

84.1% students uses mobile to take online class

#### 14. Adaptivity Level (Target Column)
> 14. Adaptivity Level : Adaptivity level of the student.

In [13]:
p_pie_plot(df['Adaptivity Level'].value_counts(), '14 Adaptivity Level (Target Column)')

Adaptability level which is our target column refer to adaptability level of the student during online education. <br>

It can be seen that about 51.9% respond they have moderate adaptability, while low accounts for 39.8% and high the rest.

## 2.2. Multivariate Analysis
> 2.2.1 Age Distribution by Gender

In [14]:
gender_age = df.groupby(['Gender' , 'Age']).size()
fig = go.Figure(data = [
    go.Bar(name = 'Boy', x = gender_age['Boy'].index, y=gender_age['Boy'].values, text=gender_age['Boy'].values, marker_color = color_scheme[0]),
    go.Bar(name = 'Girl', x=gender_age['Girl'].index, y=gender_age['Girl'].values, text=gender_age['Girl'].values, marker_color = color_scheme[1])
])

fig.update_layout(barmode = 'group', xaxis_tickangle = -45, title = 'Age Distribution by Gender', template = 'simple_white')

fig.show()

Female's age mainly distributed in 11-20 while age of male mostly distributed between 11-15 and 21-25

#### 2.2.2 IT Student Distribution by Gender

In [15]:
gender_IT = df.groupby(['Gender' , 'IT Student']).size()
fig = go.Figure(data = [
    go.Bar(name = 'Boy', x=gender_IT['Boy'].index, y=gender_IT['Boy'].values, text = gender_IT['Boy'].values, marker_color = color_scheme[0]),
    go.Bar(name = 'Girl', x=gender_IT['Girl'].index, y=gender_IT['Girl'].values, text=gender_IT['Girl'].values, marker_color = color_scheme[1])
])

fig.update_layout(barmode = 'group', xaxis_tickangle = -45, title='Distribution of IT Students by Gender', template = 'simple_white')
fig.update_traces(width =0.3)
fig.show()

Here we can notice that male tend to study as IT student compared to female

#### 2.2.3 IT Student Distribution by Education

In [17]:
Education_IT = df.groupby(['Education Level','IT Student']).size()
fig = go.Figure(data = [
    go.Bar(name = 'College', x=Education_IT['College'].index, y=Education_IT['College'].values, text = Education_IT['College'].values, marker_color = color_scheme[0]),
    go.Bar(name = 'School', x=Education_IT['School'].index, y=Education_IT['School'].values, text = Education_IT['School'].values, marker_color = color_scheme[1]),
    go.Bar(name = 'University', x=Education_IT['University'].index, y=Education_IT['University'].values, text = Education_IT['University'].values, marker_color = color_scheme[2])
])

fig.update_layout(barmode = 'group', xaxis_tickangle = -45, title = 'Distribution of IT Student by Edu', template = 'simple_white')
fig.update_traces(width=0.2)
fig.show()

It is obvious that university got the most IT students, whereas 30 and 27 IT students in college and school.

#### 2.2.4 Gender Distribution by Adaptivity Level

In [18]:
Gender_Adaptivity = df.groupby(['Gender','Adaptivity Level']).size()
fig = go.Figure(data = [
    go.Bar(name= 'Boy', x=Gender_Adaptivity['Boy'].index, y=Gender_Adaptivity['Boy'].values, text = Gender_Adaptivity['Boy'].values, marker_color = color_scheme[0]),
    go.Bar(name = 'Girl',x=Gender_Adaptivity['Girl'].index, y=Gender_Adaptivity['Girl'].values, text = Gender_Adaptivity['Girl'].values, marker_color = color_scheme[1])

])

fig.update_layout(barmode = 'group', xaxis_tickangle = -45, title = 'Distribution of Gender by Adaptivity Level', template = 'simple_white')

fig.update_traces(width = 0.3)
fig.show()

In the survey, 71 male respondents answered as they have high adaptivity level in online education compared to 29 female.

Additional, both male and female overall got moderater adaptivity level.

#### 2.2.5 Age Distribution by Adaptivity level

In [19]:
Age_Adaptivity = df.groupby(['Age','Adaptivity Level']).size()
fig = go.Figure(data=[
    go.Bar(name='1-5', x=Age_Adaptivity['1-5'].index, y=Age_Adaptivity['1-5'].values,
           text=Age_Adaptivity['1-5'].values, marker_color=color_scheme[0]),
    
    go.Bar(name='6-10', x=Age_Adaptivity['6-10'].index, y=Age_Adaptivity['6-10'].values,
           text=Age_Adaptivity['6-10'].values, marker_color=color_scheme[1]),
    
    go.Bar(name='11-15', x=Age_Adaptivity['11-15'].index, y=Age_Adaptivity['11-15'].values,
           text=Age_Adaptivity['11-15'].values, marker_color=color_scheme[2]),
    
    go.Bar(name='16-20', x=Age_Adaptivity['16-20'].index, y=Age_Adaptivity['16-20'].values,
           text=Age_Adaptivity['16-20'].values, marker_color=color_scheme[3]),
    
    go.Bar(name='21-25', x=Age_Adaptivity['21-25'].index, y=Age_Adaptivity['21-25'].values,
           text=Age_Adaptivity['21-25'].values, marker_color=color_scheme[4]),
    
    go.Bar(name='26-30', x=Age_Adaptivity['26-30'].index, y=Age_Adaptivity['26-30'].values,
           text=Age_Adaptivity['26-30'].values, marker_color=color_scheme[5]),
])
fig.update_layout(barmode='group', xaxis_tickangle=-45,title='Distribution of Age by Adaptivity Level',
                  template = 'simple_white')

fig.update_traces(width=0.1)

fig.show()

11-15, 21-25 and 26-300 relatively got high adaptivity level while obvious that none of 1-5 got high adaptivity

## 3. Preprocessing

In [20]:
df.isnull().sum()

Gender                 0
Age                    0
Education Level        0
Institution Type       0
IT Student             0
Location               0
Load-shedding          0
Financial Condition    0
Internet Type          0
Network Type           0
Class Duration         0
Self Lms               0
Device                 0
Adaptivity Level       0
dtype: int64

In [21]:
df.isna().sum()

Gender                 0
Age                    0
Education Level        0
Institution Type       0
IT Student             0
Location               0
Load-shedding          0
Financial Condition    0
Internet Type          0
Network Type           0
Class Duration         0
Self Lms               0
Device                 0
Adaptivity Level       0
dtype: int64

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Gender               1205 non-null   object
 1   Age                  1205 non-null   object
 2   Education Level      1205 non-null   object
 3   Institution Type     1205 non-null   object
 4   IT Student           1205 non-null   object
 5   Location             1205 non-null   object
 6   Load-shedding        1205 non-null   object
 7   Financial Condition  1205 non-null   object
 8   Internet Type        1205 non-null   object
 9   Network Type         1205 non-null   object
 10  Class Duration       1205 non-null   object
 11  Self Lms             1205 non-null   object
 12  Device               1205 non-null   object
 13  Adaptivity Level     1205 non-null   object
dtypes: object(14)
memory usage: 131.9+ KB


In [28]:
# Get column name of dataframe
# columns = ['Gender', 'Age','Education Level', 'Institution Type','IT Student','Location','Load-shedding','Financail Condition','Internet Type','Network Type','Class Duration',
#            'Self Lms','Device','Adaptivity Level']
columns = ['Gender','Age','Education Level','Institution Type','IT Student','Location',
           'Load-shedding','Financial Condition','Internet Type','Network Type',
           'Class Duration','Self Lms','Device','Adaptivity Level']

In [29]:
for column in columns:
  label_encoder = LabelEncoder()
  df[column] = label_encoder.fit_transform(df[column])

In [30]:
df.head()

,Gender,Age,Education Level,Institution Type,IT Student,Location,Load-shedding,Financial Condition,Internet Type,Network Type,Class Duration,Self Lms,Device,Adaptivity Level
0,0,3,2,1,0,1,1,0,1,2,2,0,2,2
1,1,3,2,1,0,1,0,0,0,2,1,1,1,2
2,1,2,0,0,0,1,1,0,1,2,1,0,1,2
3,1,1,1,1,0,1,1,0,0,2,1,0,1,2
4,1,2,1,1,0,1,1,1,0,1,0,0,1,1


## 3.2 Train Test Split

split dataframe into train and test set

In [31]:
y = df['Adaptivity Level']
x = df.drop('Adaptivity Level', axis = 1)
x_train, x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3, random_state=116)

In [33]:
print(colored('Shape of x_train:','blue'), x_train.shape,'\n')
print(colored('Shape of x_test:','red'), x_test.shape,'\n')

Shape of x_train (843, 13) 

Shape of x_test (362, 13) 



## 4.0 Modeling

#### 4.1 Define Preprocessing & Mideling functions

In [34]:
from yellowbrick.contrib.wrapper import CLASSIFIER
class CONFIG:
  FOLD = 5
  RANDOM_STATE = 116
  # 4 models

  CLASSIFIERS = {
      "LogisticRegression":LogisticRegression(max_iter = 1000, random_state = RANDOM_STATE),
      "KNearest": KNeighborsClassifier(),
      "RandomForestClassifier": RandomForestClassifier(random_state=RANDOM_STATE),
      "XGBClassfier": XGBClassifier(random_state = RANDOM_STATE),
      "CatBoostClassifier":CatBoostClassifier(random_state = RANDOM_STATE, logging_level = 'Silent')

  }

  FEATURES = ['Gender','Age','Education Level','Institution Type','IT Student','Location',
           'Load-shedding','Financial Condition','Internet Type','Network Type',
           'Class Duration','Self Lms','Device']

# catboost Best parameter (using optuna)

  CB_BEST_PARAMS = {'iterations':815,'depth':5,'learning_rate':0.0861677, 'random_strength': 45, 'bagging_temperature':0.2394,
                  'od_type':'Iter','od_wait':18,
                  'logging_level':'Silent',
                  'random_state': RANDOM_STATE }

                  # XGBoost Best Parameters (using Optuna)
  XGB_BEST_PARAMS = {'n_estimators':634, 'max_depth':15, 'reg_alpha':0, 'reg_lambda':3, 'min_child_weight': 0,
                     'gamma':0, 'learning_rate': 0.2616,
                     'colsample_bytree':0.65,
                     'random_state':RANDOM_STATE}


In [42]:
# scaler function
def _scale(train_data, val_data, features):
  scaler = StandardScaler()
  scaled_train = scaler.fit_transform(train_data[features])
  scaled_val = scaler.transform(val_data[features])

  train = train_data.copy()
  val = val_data.copy()

  train[features] = scaled_train
  val[features] = scaled_val

  return train, val

In [45]:
def classifiers_modeling(classifiers, x_train, y_train_, x_test,y_test, features):
  accuracy_list = []
  classifiers_name = list(classifiers.keys())
  # 5 fold

  fold = KFold(n_splits = CONFIG.FOLD, random_state = CONFIG.RANDOM_STATE, shuffle = True)

  for idx, classifier in enumerate(classifiers.values()):
    accuracy = 0
    for fold_idx, (train_idx, val_idx) in enumerate(fold.split(x_train, y_train_)):
      X_train, X_val = x_train.iloc[train_idx], x_train.iloc[val_idx]
      y_train, y_val = y_train_.iloc[train_idx], y_train_.iloc[val_idx]
      X_train, X_val = _scale(X_train, X_val, features)
      model = classifier.fit(X_train[features], y_train)
      val_preds = model.predict(X_val[features])
      accuracy += accuracy_score(y_val, val_preds)/CONFIG.FOLD
    accuracy_list.append(round(accuracy,5))

    print('(', idx+1,')', classifiers_name[idx], 'cross validation (5 fold)')
    print('Mean Accuracy Score:', round(accuracy, 5))
    print('Mean Accuracy Score:', colored(round(accuracy,5)))

  return accuracy_list

## Model Comparison (Part 1)

In [46]:
classifiers_name = list(CONFIG.CLASSIFIERS.keys())
classifiers_accuracy = classifiers_modeling(CONFIG.CLASSIFIERS, x_train,y_train,x_test,y_test, CONFIG.FEATURES)

( 1 ) LogisticRegression cross validation (5 fold)
Mean Accuracy Score: 0.68809
Mean Accuracy Score: 0.68809
( 2 ) KNearest cross validation (5 fold)
Mean Accuracy Score: 0.77112
Mean Accuracy Score: 0.77112
( 3 ) RandomForestClassifier cross validation (5 fold)
Mean Accuracy Score: 0.89568
Mean Accuracy Score: 0.89568
( 4 ) XGBClassfier cross validation (5 fold)
Mean Accuracy Score: 0.77107
Mean Accuracy Score: 0.77107
( 5 ) CatBoostClassifier cross validation (5 fold)
Mean Accuracy Score: 0.88619
Mean Accuracy Score: 0.88619


In [48]:
# Model Score Visualization
p_bar_plot(x = classifiers_name, 
           y = np.round(classifiers_accuracy,3),
           width = 0.5,
           x_axis_title = 'Classifiers',
           y_axis_title = 'Classifiers Accuracy Score',
           title = 'Classifiers Accuracy Score (Cross Validated: 5 fold)'
               )

## Model Comparision (Part 2)

In [50]:
tuned_classifiers = {
    "CatBoostCAlssifier": CatBoostClassifier(**CONFIG.CB_BEST_PARAMS),
    "XGBClassifier": XGBClassifier(**CONFIG.XGB_BEST_PARAMS)
}

In [51]:
classifiers_name.append('Tuned_CatBoostClassifier')
classifiers_name.append('Tuned_XGBClassifier')

tuned_accuracy = classifiers_modeling(tuned_classifiers,x_train,y_train,x_test,y_test,CONFIG.FEATURES)
for tuned_accuracy_score in tuned_accuracy:
  classifiers_accuracy.append(tuned_accuracy_score)

( 1 ) CatBoostCAlssifier cross validation (5 fold)
Mean Accuracy Score: 0.89806
Mean Accuracy Score: 0.89806
( 2 ) XGBClassifier cross validation (5 fold)
Mean Accuracy Score: 0.89687
Mean Accuracy Score: 0.89687


In [52]:
p_bar_plot(x = classifiers_name, 
           y = np.round(classifiers_accuracy,3),
           width = 0.6,
           x_axis_title = 'Classifiers',
           y_axis_title = 'Classifiers Accuracy',
           title = 'Comparison of each Classifiers Accuracy Score (Fold=5, Tuned Model Added) ')